Action1	"文本抄袭自动检测分析:
如果你是某新闻单位工作人员（这里假设source=新华社），为了防止其他媒体抄袭你的文章，你打算做一个抄袭自动检测分析的工具
1）定义可能抄袭的文章来源
2）与原文对比定位抄袭的地方
原始数据：sqlResult.csv，共计89611篇
从数据库导出的文章，字段包括：id, author, source, content, feature, title, url
常用中文停用词：chinese_stopwords.txt"				"
1、完成聚类算法（10points）
2、完成分类算法（10points）
3、完成相似度计算，Top-N排序（10points）
4、完成代码，结果正确（30points）"

文本抄袭自动检测：
Step1，数据加载
加载sqlResult.csv及停用词chinese_stopwords.txt
Step2，数据预处理
1）数据清洗，针对content字段为空的情况，进行dropna
2）分词，使用jieba进行分词
3）将处理好的分词保存到 corpus.pkl，方便下次调用
4）数据集切分，70%训练集，30%测试集
Step3，提取文本特征TF-IDF
Step4，预测文章风格是否和自己一致
使用分类模型（比如MultinomialNB），对于文本的特征（比如TF-IDF）和label（是否为新华社）进行训练
Step5，找到可能Copy的文章，即预测label=1，但实际label=0
Step6，根据模型预测的结果来对全量文本进行比对，如果数量很大，我们可以先用k-means进行聚类降维，比如k=25种聚类
Step7，找到一篇可能的Copy文章，从相同label中，找到对应新华社的文章，并按照TF-IDF相似度矩阵，从大到小排序，取Top10
Step8，使用编辑距离editdistance，计算两篇文章的距离
Step9，精细比对，对于疑似文章与原文进行逐句比对，即计算每个句子的编辑距离editdistance

In [ ]:
import pandas as pd
import numpy as np
import jieba

In [ ]:
#数据加载
news = pd.read_csv('sqlResult.csv',encoding='gb18030')
print(news.shape)
print(news.head())

In [ ]:
news[news.content.isna()].head()
news = news.dropna(subset = ['content'])
print(news.shape)

In [ ]:
#加载停用词
with open('chinese_stopwords.txt','r',encoding = 'utf-8') as file:
    stopwords = [i[:-1] for i in file.readlines()]
    

#分词
def split_text(text):
    text  = text.replace(' ','').replace('\n','')
    text2 = jieba.cut(text)
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

In [ ]:
print(news.iloc[0].content)

In [ ]:
split_text(news.iloc[0].content)

In [ ]:
corpus = list(map(split_text,[str(i) for i in news.content]))

In [ ]:
corpus

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
#计算corpus的TFIDF（文本特征）
countvectorizer = CountVectorizer(encoding = 'gb18030', min_df = 0.015)
tfidftransformer = TfidfTransformer()

countvector = countvectorizer.fit_transform(corpus)
tfidf = tfidftransformer.fit_transform(countvector)
print(tfidf.shape)

In [ ]:
#标记是否为自己的新闻
label = list(map(lambda source:1 if '新华社' in str(source) else 0,news.source))
label

In [96]:
from sklearn.model_selection import train_test_split
#切分数据集
x_train,x_test,y_train,y_test = train_test_split(tfidf.toarray(),label,test_size = 0.3)


In [97]:
#
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
model = MultinomialNB()
model.fit(x_train,y_train)

model2 = BernoulliNB()
model2.fit(x_train,y_train)

BernoulliNB()

In [98]:
from sklearn.metrics import accuracy_score, precision_score,recall_score
y_predict = model.predict(x_test)

print('准确率：',accuracy_score(y_test,y_predict))
print('精确度：',precision_score(y_test,y_predict))
print('召回率：',recall_score(y_test,y_predict))

准确率： 0.8802695562277444
精确度： 0.961746906331858
召回率： 0.903287380699894


In [99]:
y2_predict = model2.predict(x_test)
print('准确率：',accuracy_score(y_test,y2_predict))
print('精确度：',precision_score(y_test,y2_predict))
print('召回率：',recall_score(y_test,y2_predict))

准确率： 0.8245204273078838
精确度： 0.9660384766391834
召回率： 0.8349522799575821


In [100]:
#使用模型进行风格预测
prediction = model.predict(tfidf.toarray())
labels = np.array(label)

#prediction为预测风格，labels为真实结果
compare_news_index = pd.DataFrame({'prediction':prediction,'labels':labels})
copy_news_index = compare_news_index[(compare_news_index['prediction'] == 1) & (compare_news_index['labels'] == 0)].index

In [101]:
#统计实际为新华社的新闻
xhs_news_index = compare_news_index[(compare_news_index['labels'] == 1)].index

In [102]:
print('可能为抄袭的新闻条数：',len(copy_news_index))

可能为抄袭的新闻条数： 2794


In [103]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfidf.toarray())

#使用Kmeans进行全量文档进行聚类
kmeans = KMeans(n_clusters = 25)
k_labels = kmeans.fit_predict(scaled_array)
print(k_labels.shape)

(87054,)


In [104]:
#创建id_class,ID(1-8700+),class(1-25)索引
id_class = {index:class_ for index, class_ in enumerate(k_labels)}
from collections import defaultdict
class_id = defaultdict(set)

for index,class_ in id_class.items():
    #只统计新华社发布的class_id
    if index in xhs_news_index.tolist():
        class_id[class_].add(index)

In [105]:
from sklearn.metrics.pairwise import cosine_similarity
#查找相似文章
def find_similar_text(cpindex,top=10):
    #只在新华社发布得的文章找
    dist_dist = {i:cosine_similarity(tfidf[cpindex],tfidf[i])for i in class_id[id_class[cpindex]]}
    #从大到小排序
    return sorted(dist_dist.items(),key=lambda x:x[1][0],reverse =True)[:top]

In [106]:
cpindex = 3352
print('是否在新华社',cpindex in xhs_news_index)
print('是否在抄袭新闻里',cpindex in copy_news_index)

是否在新华社 False
是否在抄袭新闻里 True


In [107]:
similar_list = find_similar_text(cpindex)
print(similar_list)

[(3134, array([[0.96849134]])), (63511, array([[0.94643198]])), (29441, array([[0.94283416]])), (3218, array([[0.87621892]])), (29615, array([[0.86936328]])), (29888, array([[0.86215862]])), (64046, array([[0.85278235]])), (29777, array([[0.84875422]])), (63974, array([[0.73415212]])), (63975, array([[0.73415212]]))]


In [108]:
print('怀疑抄袭：\n',news.iloc[cpindex].content)
#找一篇相似的文章
similar2 = similar_list[0][0]
print('相似的原文：\n',news.iloc[similar2].content)

怀疑抄袭：
 　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。
　　统计局：15个一线和热点二线城市房价同比涨幅全部回落
　　国家统计局城市司高级统计师刘建伟解读5月份房价数据
　　5月份一二线城市房价平均涨幅继续回落
　　国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。
　　一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平
　　5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。
　　二、70个大中城市中一二线城市房价同比涨幅持续回落
　　5月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。
　　三、70个大中城市中房价环比下降及涨幅回落城市个数均有所增加
　　5月份，70个城市中新建商品住宅价格环比下降的城市有9个，比上月增加1个；涨幅回落的城市有26个，比上月增加3个。二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个。

相似的原文：
 　　国家统计局19日发布数据，5月份，15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落，其中9个城市环比下降或持平。这9个价格环比下降或持平的城市为：北京、上海、南京、杭州、合肥、福州、郑州、深圳、成都。
　　“5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。”国家统计局城市司高级统计师刘建伟说，从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新